<a href="https://colab.research.google.com/github/ejpark78/codelab/blob/master/%5B%EA%B3%B5%EA%B0%9C%EC%9A%A9%5D_Colab%EC%97%90%EC%84%9C_TPU%EB%A1%9C_KcBERT_%EC%B2%98%EC%9D%8C%EB%B6%80%ED%84%B0_Pretrain%ED%95%98%EA%B8%B0_with_Korpora.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## 준비물

- GCS버킷 사용이 활성화된 GCP 계정이 있어야 합니다.
- https://console.cloud.google.com/?hl=ko
- 위 링크에 들어가서 가입해주세요.
- 원하는 이름으로 GCS 버킷을 만들어두세요.
- 팁: MultiRegion US로 만드는게 학습 속도가 빨라요.
- https://console.cloud.google.com/storage/browser

- 구글 드라이브 마운트 (화면 좌측 '파일' 내 구글드라이브 아이콘 클릭)

<!-- 가입 후 아래 링크에 들어가서 "서비스 키"를 만들고, 다운받은 json을 **Colab에 업로드**해주세요. -->

<!-- 그리고 아래에 그 파일 이름을 적어주셔야 합니다. -->

<!-- https://console.cloud.google.com/iam-admin/serviceaccounts -->

Tensorflow 1.x (1.15) 환경을 활성화해 줍니다.

2020년 09월 11일 기준, Colab은 기본적으로 TF2 환경입니다.

In [ ]:
%tensorflow_version 1.x

TensorFlow 1.x selected.


In [ ]:
!mkdir /content/drive/My\ Drive/NLP_EXAMPLE/
!mkdir /content/drive/My\ Drive/NLP_EXAMPLE/KcBERT-Korpora
%cd /content/drive/My\ Drive/NLP_EXAMPLE/KcBERT-Korpora

mkdir: cannot create directory ‘/content/drive/My Drive/NLP_EXAMPLE/’: File exists
mkdir: cannot create directory ‘/content/drive/My Drive/NLP_EXAMPLE/KcBERT-Korpora’: File exists
/content/drive/My Drive/NLP_EXAMPLE/KcBERT-Korpora


In [ ]:
!pip install -q Korpora
!pip install -q tokenizers
!git clone -q https://github.com/google-research/bert

     |████████████████████████████████| 51kB 1.9MB/s 
     |████████████████████████████████| 71kB 4.0MB/s 
     |████████████████████████████████| 3.0MB 3.5MB/s 


In [ ]:
import os
import sys
import json
import nltk
import random
import logging
import tensorflow as tf

from glob import glob
from google.colab import auth, drive
from tensorflow.keras.utils import Progbar

sys.path.append("bert")

from bert import modeling, optimization, tokenization
from bert.run_pretraining import input_fn_builder, model_fn_builder

아래 셀을 실행하면 링크가 나옵니다.

링크를 클릭후 앞서 사용한 GCS 로그인 되는 계정을 클릭한 뒤, 거기서 나오는 토큰값을 복사해 아래 입력값에 넣어주세요.

In [ ]:
auth.authenticate_user()

The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.



In [ ]:
# configure logging
log = logging.getLogger('tensorflow')
log.setLevel(logging.INFO)

# create formatter and add it to the handlers
formatter = logging.Formatter('%(asctime)s :  %(message)s')
sh = logging.StreamHandler()
sh.setLevel(logging.INFO)
sh.setFormatter(formatter)
log.handlers = [sh]

if 'COLAB_TPU_ADDR' in os.environ:
  log.info("Using TPU runtime")
  USE_TPU = True
  TPU_ADDRESS = 'grpc://' + os.environ['COLAB_TPU_ADDR']

  with tf.Session(TPU_ADDRESS) as session:
    log.info('TPU address is ' + TPU_ADDRESS)
    # Upload credentials to TPU.
    with open('/content/adc.json', 'r') as f:
      auth_info = json.load(f)
    tf.contrib.cloud.configure_gcs(session, credentials=auth_info)
    
else:
  raise Exception("TPU환경이 아닙니다. 런타임 옵션에서 TPU환경으로 바꿔주세요.")

2020-09-11 03:04:20,869 :  Using TPU runtime
2020-09-11 03:04:20,871 :  TPU address is grpc://10.69.133.74:8470


In [ ]:
from Korpora import Korpora

Korpora.fetch('kcbert', root_dir='.')

[kcbert] download kcbert-train.tar.gzaa: 100%|██████████| 2.10G/2.10G [00:36<00:00, 57.7MB/s]
[kcbert] download kcbert-train.tar.gzab: 100%|██████████| 2.10G/2.10G [00:27<00:00, 76.0MB/s]
[kcbert] download kcbert-train.tar.gzac: 671MB [00:14, 45.1MB/s]                           


Unzip tar. It needs a few minutes ... done


In [ ]:
!ls kcbert/

20190101_20200611_v2.txt  kcbert-train.tar.gzab
kcbert-train.tar.gzaa	  kcbert-train.tar.gzac


실제 KcBERT 학습을 진행한 데이터셋 크기는 약 12GB로, Colab 실행 시간제약(Pro기준 24시간, 일반 12시간) 내에 학습을 절대 끝낼 수 없다.

따라서 약 9천만개 문장 중 100만개 정도의 문장만 골라 학습해보자.

In [ ]:
DEMO_MODE = True #@param {type:"boolean"}

if DEMO_MODE:
    CORPUS_SIZE = 1000000 #@param {type:"number"}
    !(head -n $CORPUS_SIZE kcbert/20190101_20200611_v2.txt) > dataset.txt  
else:
    !mv kcbert/20190101_20200611_v2.txt dataset.txt

In [ ]:
!ls -lh dataset.txt # 약 144MB

-rw------- 1 root root 144M Sep 11 03:07 dataset.txt


위 데이터셋은 이미 전처리를 끝낸 데이터셋이다. 따라서 별도의 전처리는 필요하지 않다.

데이터셋을 적당한 사이즈로 잘라주자. (이후 Tokenizer등 좀더 빠르게 처리할 수 있다.)

100만개를 선택했다면 4개의 파일이 생성된다.

In [ ]:
!mkdir ./shards
!split -a 4 -l 256000 -d dataset.txt ./shards/shard_
!ls ./shards/

shard_0000  shard_0001	shard_0002  shard_0003


이제 Huggingface의 Tokenizers 라이브러리를 통해 Tokenizer를 만들어보자.

다만 크기를 고려해 적당히 작은 크기의 Vocab을 만들어보자.

In [ ]:
from tokenizers import BertWordPieceTokenizer

In [ ]:
tokenizer = BertWordPieceTokenizer(lowercase=False)

Vocab 사이즈는 1만 + Alphabet(단일글자) 최대 3000 + BERT문자 4개 = 총 13,004개

In [ ]:
tokenizer.train(['dataset.txt'], vocab_size=13004, limit_alphabet=3000)

현재 폴더의 `vocab.txt` 파일로 토크나이저를 저장해준다.

In [ ]:
tokenizer.save_model('.')

['./vocab.txt']

토크나이저가 정상적으로 학습된 것을 볼 수 있다.

In [ ]:
!tail -n 10 vocab.txt

소리는
자진
쪽발
철면피
하던지
##가격
##제동
##보인다
##안하는
##와는


이제 어떤 설정의 BERT를 학습할 것인지 정해야 한다.

- MAX_SEQ_LENGTH: BERT의 모델 입력의 최장 토큰 길이
이 이상으로는 BERT모델이 이해하지 못한다.
- MASKED_LM_PROB: BERT의 학습 중 Masked LM의 비율을 조정한다.
- MAX_PREDICTIONS: Sequence별 예측할 최대 숫자
- DO_LOWER_CASE: 영문자를 lower(소문자화) 할 지. 
- PROCESSES: 전처리할때 CPU 몇개 쓸지
- PRETRAINING_DIR: 프리트레인 데이터 폴더 이름
- VOC_FNAME: 방금 만들어준 vocab.txt 위치

In [ ]:
MAX_SEQ_LENGTH = 128 #@param {type:"integer"}
MASKED_LM_PROB = 0.15 #@param
MAX_PREDICTIONS =  20#@param {type:"integer"}
DO_LOWER_CASE = False #@param {type:"boolean"}
PROCESSES = 4 #@param {type:"integer"}
PRETRAINING_DIR = "pretraining_data" #@param {type:"string"}
VOC_FNAME = 'vocab.txt' #@param {type:"string"}

아래 명령어를 실행하면 약 5~15분 사이로 작업이 완료된다.

In [ ]:
XARGS_CMD = ("ls ./shards/ | "
             "xargs -n 1 -P {} -I{} "
             "python3 bert/create_pretraining_data.py "
             "--input_file=./shards/{} "
             "--output_file={}/{}.tfrecord "
             "--vocab_file={} "
             "--do_lower_case={} "
             "--max_predictions_per_seq={} "
             "--max_seq_length={} "
             "--masked_lm_prob={} "
             "--random_seed=34 "
             "--dupe_factor=5")

XARGS_CMD = XARGS_CMD.format(PROCESSES, '{}', '{}', PRETRAINING_DIR, '{}', 
                             VOC_FNAME, DO_LOWER_CASE, 
                             MAX_PREDICTIONS, MAX_SEQ_LENGTH, MASKED_LM_PROB)

tf.gfile.MkDir(PRETRAINING_DIR)
!$XARGS_CMD







W0911 03:08:32.682168 140213738506112 module_wrapper.py:139] From bert/create_pretraining_data.py:437: The name tf.logging.set_verbosity is deprecated. Please use tf.compat.v1.logging.set_verbosity instead.
W0911 03:08:32.682178 140601750513536 module_wrapper.py:139] From bert/create_pretraining_data.py:437: The name tf.logging.set_verbosity is deprecated. Please use tf.compat.v1.logging.set_verbosity instead.




W0911 03:08:32.682237 140404938434432 module_wrapper.py:139] From bert/create_pretraining_data.py:437: The name tf.logging.set_verbosity is deprecated. Please use tf.compat.v1.logging.set_verbosity instead.
W0911 03:08:32.682201 140021592127360 module_wrapper.py:139] From bert/create_pretraining_data.py:437: The name tf.logging.set_verbosity is deprecated. Please use tf.compat.v1.logging.set_verbosity instead.




W0911 03:08:32.682343 140601750513536 module_wrapper.py:139] From bert/create_pretraining_data.py:437: The name tf.logging.INFO is deprecated. Please use tf.c

Colab의 TPU를 100% 활용하기 위해서는 GCS 버킷과 통신하는 것이 가장 좋다. (사실 아직 local과 통신 지원을 안한다 ㅠㅠ)

따라서 해당 GCS Bucket을 만들고 아래 `BUCKET_NAME`에 적어주자.

> 팁: Colab 특성상 US 내부 모든 GCP 리전을 사용하기 때문에 MultiRegion-US를 선택하는 것이 학습 속도에 가장 유리하다.

GCS 버킷 콘솔: https://console.cloud.google.com/storage/browser


---


# 주의: BUCKET_NAME을 꼭 바꿔주세요!

In [ ]:
BUCKET_NAME = "" #@param {type:"string"}
MODEL_DIR = "bert_model" #@param {type:"string"}
tf.gfile.MkDir(MODEL_DIR)

assert BUCKET_NAME != ''

학습할 BERT 설정을 해준다.

아래 설정은 BERT-Base모델과 동일하다. (단, Token size와 Max Length는 다르다.)

In [ ]:
bert_base_config = {
  "attention_probs_dropout_prob": 0.1, 
  "directionality": "bidi", 
  "hidden_act": "gelu", 
  "hidden_dropout_prob": 0.1, 
  "hidden_size": 768, 
  "initializer_range": 0.02, 
  "intermediate_size": 3072, 
  "max_position_embeddings": 512, 
  "num_attention_heads": 12, 
  "num_hidden_layers": 12, 
  "pooler_fc_size": 768, 
  "pooler_num_attention_heads": 12, 
  "pooler_num_fc_layers": 3, 
  "pooler_size_per_head": 128, 
  "pooler_type": "first_token_transform", 
  "type_vocab_size": 2, 
  "vocab_size": 13004, # <- 원래는 3만개+, 우리는 1만3천개
}

with open("{}/bert_config.json".format(MODEL_DIR), "w") as fo:
  json.dump(bert_base_config, fo, indent=2)
  
!cp vocab.txt $MODEL_DIR

In [ ]:
!ls $MODEL_DIR

bert_config.json  vocab.txt


앞서 만들어준 학습용 데이터, Bert Config json파일, 그리고 Vocab을 모두 GCS 버킷에 업로드한다.

In [ ]:
if BUCKET_NAME:
    !gsutil -m cp -r $MODEL_DIR $PRETRAINING_DIR gs://$BUCKET_NAME

Copying file://bert_model/bert_config.json [Content-Type=application/json]...
Copying file://bert_model/vocab.txt [Content-Type=text/plain]...
Copying file://pretraining_data/shard_0000.tfrecord [Content-Type=application/octet-stream]...
==> NOTE: You are uploading one or more large file(s), which would run
significantly faster if you enable parallel composite uploads. This
feature can be enabled by editing the
"parallel_composite_upload_threshold" value in your .boto
configuration file. However, note that if you do this large files will
be uploaded as `composite objects
<https://cloud.google.com/storage/docs/composite-objects>`_,which
means that any user who downloads such objects will need to have a
compiled crcmod installed (see "gsutil help crcmod"). This is because
without a compiled crcmod, computing checksums on composite objects is
so slow that gsutil disables downloads of composite objects.

Copying file://pretraining_data/shard_0001.tfrecord [Content-Type=application/octet-st

업로드가 완료되면 학습을 진행할 설정을 잡아주자.

앞서 잡아준것과 동일하고, Train step은 100만으로 설정하면 딱 1epoch를 학습할 수 있게 된다.

또한, 2500step별로 GCS에 저장해 이어서 학습할 수 있도록 만들어준다.

In [ ]:
# Input data pipeline config
TRAIN_BATCH_SIZE = 128 #@param {type:"integer"}
MAX_PREDICTIONS = 20 #@param {type:"integer"}
MAX_SEQ_LENGTH = 128 #@param {type:"integer"}
MASKED_LM_PROB = 0.15 #@param

# Training procedure config
EVAL_BATCH_SIZE = 64
LEARNING_RATE = 2e-5
TRAIN_STEPS = 1000000 #@param {type:"integer"}
SAVE_CHECKPOINTS_STEPS = 2500 #@param {type:"integer"}
NUM_TPU_CORES = 8

if BUCKET_NAME != '.':
  BUCKET_PATH = "gs://{}".format(BUCKET_NAME)
else:
  BUCKET_PATH = "."

BERT_GCS_DIR = "{}/{}".format(BUCKET_PATH, MODEL_DIR)
DATA_GCS_DIR = "{}/{}".format(BUCKET_PATH, PRETRAINING_DIR)

VOCAB_FILE = os.path.join(BERT_GCS_DIR, VOC_FNAME)
CONFIG_FILE = os.path.join(BERT_GCS_DIR, "bert_config.json")

INIT_CHECKPOINT = tf.train.latest_checkpoint(BERT_GCS_DIR)

bert_config = modeling.BertConfig.from_json_file(CONFIG_FILE)
input_files = tf.gfile.Glob(os.path.join(DATA_GCS_DIR,'*tfrecord'))

log.info("Using checkpoint: {}".format(INIT_CHECKPOINT))
log.info("Using {} data shards".format(len(input_files)))

2020-09-11 03:14:06,058 :  From /content/drive/My Drive/NLP_EXAMPLE/KcBERT-Korpora/bert/modeling.py:93: The name tf.gfile.GFile is deprecated. Please use tf.io.gfile.GFile instead.

2020-09-11 03:14:06,217 :  Using checkpoint: None
2020-09-11 03:14:06,218 :  Using 4 data shards


GCS에 있는 데이터를 Tensorflow가 가져다 학습하려면 TPU 위치와 GCS위치를 지정해주면 된다.

In [ ]:
model_fn = model_fn_builder(
      bert_config=bert_config,
      init_checkpoint=INIT_CHECKPOINT,
      learning_rate=LEARNING_RATE,
      num_train_steps=TRAIN_STEPS,
      num_warmup_steps=10000,
      use_tpu=USE_TPU,
      use_one_hot_embeddings=True)

tpu_cluster_resolver = tf.contrib.cluster_resolver.TPUClusterResolver(TPU_ADDRESS)

run_config = tf.contrib.tpu.RunConfig(
    cluster=tpu_cluster_resolver,
    model_dir=BERT_GCS_DIR,
    save_checkpoints_steps=SAVE_CHECKPOINTS_STEPS,
    tpu_config=tf.contrib.tpu.TPUConfig(
        iterations_per_loop=SAVE_CHECKPOINTS_STEPS,
        num_shards=NUM_TPU_CORES,
        per_host_input_for_training=tf.contrib.tpu.InputPipelineConfig.PER_HOST_V2))

estimator = tf.contrib.tpu.TPUEstimator(
    use_tpu=USE_TPU,
    model_fn=model_fn,
    config=run_config,
    train_batch_size=TRAIN_BATCH_SIZE,
    eval_batch_size=EVAL_BATCH_SIZE)
  
train_input_fn = input_fn_builder(
        input_files=input_files,
        max_seq_length=MAX_SEQ_LENGTH,
        max_predictions_per_seq=MAX_PREDICTIONS,
        is_training=True)

2020-09-11 03:14:07,231 :  Estimator's model_fn (<function model_fn_builder.<locals>.model_fn at 0x7f23a2176ae8>) includes params argument, but params are not passed to Estimator.
2020-09-11 03:14:07,236 :  Using config: {'_model_dir': 'gs://colab-kcbert-tutorial-2/bert_model', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': 2500, '_save_checkpoints_secs': None, '_session_config': allow_soft_placement: true
cluster_def {
  job {
    name: "worker"
    tasks {
      key: 0
      value: "10.69.133.74:8470"
    }
  }
}
isolate_session_state: true
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': None, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_experimental_max_worker_delay_secs': None, '_session_creation_timeout_secs': 7200, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7f23a216

# 이제 학습을 진행해보자!

> 만약 TPU 에러가 난다면, Bucket을 `allUser` R/W 권한을 줘야 합니다. (구글 코랩 문제임)

In [ ]:
# 학습하자!!
estimator.train(input_fn=train_input_fn, max_steps=TRAIN_STEPS)

2020-09-11 03:14:07,332 :  Querying Tensorflow master (grpc://10.69.133.74:8470) for TPU system metadata.
2020-09-11 03:14:07,348 :  Found TPU system:
2020-09-11 03:14:07,349 :  *** Num TPU Cores: 8
2020-09-11 03:14:07,350 :  *** Num TPU Workers: 1
2020-09-11 03:14:07,350 :  *** Num TPU Cores Per Worker: 8
2020-09-11 03:14:07,350 :  *** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:CPU:0, CPU, -1, 7203291503498404454)
2020-09-11 03:14:07,352 :  *** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU:0, TPU, 17179869184, 6381240814083094157)
2020-09-11 03:14:07,352 :  *** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU:1, TPU, 17179869184, 2410460841412794417)
2020-09-11 03:14:07,352 :  *** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU:2, TPU, 17179869184, 3518060784351875248)
2020-09-11 03:14:07,353 :  *** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TP

2020-09-11 03:14:08,065 :  From bert/modeling.py:671: dense (from tensorflow.python.layers.core) is deprecated and will be removed in a future version.
Instructions for updating:
Use keras.layers.Dense instead.
2020-09-11 03:14:08,067 :  From /tensorflow-1.15.2/python3.6/tensorflow_core/python/layers/core.py:187: Layer.apply (from tensorflow.python.keras.engine.base_layer) is deprecated and will be removed in a future version.
Instructions for updating:
Please use `layer.__call__` method instead.
2020-09-11 03:14:10,606 :  From /content/drive/My Drive/NLP_EXAMPLE/KcBERT-Korpora/bert/run_pretraining.py:150: The name tf.trainable_variables is deprecated. Please use tf.compat.v1.trainable_variables instead.

2020-09-11 03:14:10,607 :  **** Trainable Variables ****
2020-09-11 03:14:10,607 :    name = bert/embeddings/word_embeddings:0, shape = (13004, 768)
2020-09-11 03:14:10,608 :    name = bert/embeddings/token_type_embeddings:0, shape = (2, 768)
2020-09-11 03:14:10,608 :    name = bert/e

위 학습을 진행 중, GCS 웹 콘솔에서 'Cloud Shell' 기능을 통해 Tensorboard로 loss를 추적할 수 있습니다.

아래와 같이 `bert_model` 폴더가 있는 경로를 지정해 tensorboard를 띄우고, '웹 미리보기' 기능을 이용하면 아래와 같이 텐서보드를 볼 수 있습니다.

```shell
#tensorboard --logdir gs://colab-kcbert-tutorial-2/bert_model
tensorboard --logdir gs://버킷이름/bert_model
```

아래 스크린샷처럼 loss가 감소하는 것을 볼 수 있습니다.

![텐서보드 로그](https://i.imgur.com/0JVSVuj.png)